In [11]:
import pandas as pd
import glob
import math
import numpy as np

In this notebook we show to locally evaluate a submission generated from a subset of the csv's from your training data.

In [12]:
data_path = 'DATASET-PATH' # TODO: point this to your data folder
submission_path = data_path + 'submissions/'
training_path = data_path + 'training_set/'
input_logs = sorted(glob.glob(training_path + "log_mini*.csv")) # TODO: point this to a subset of csv's in your training set

In [13]:
def get_ground_truth(input_logs):
    ground_truths = []     
    for i,f in enumerate(input_logs):
        df = pd.read_csv(f)
        # Below we keep only the relevant columns of the second half of the session for saving the ground truth
        df = df[['session_id','skip_2','session_position','session_length']].loc[df['session_position']*2 > df['session_length']]
        df = df.reset_index()
        current_index = 0
        # Here we process each session, saving a list containing the 
        while current_index < len(df):
            partial_length = df['session_length'].iloc[current_index]-df['session_position'].iloc[current_index]+1
            session_skips = list(df.loc[current_index:current_index+partial_length-1, 'skip_2'])
            ground_truths.append(session_skips)
            current_index += partial_length 
    return ground_truths

In [14]:
ground_truth = get_ground_truth(input_logs)

In [15]:
def generate_random_submission(input_logs):
    output = []
    for i,f in enumerate(input_logs):
        df = pd.read_csv(f)
        print('file {} read'.format(i))
        # For this random submission we just need to know the length of each session
        df = df.drop_duplicates(subset='session_id', keep='first', inplace=False)
        # For each session we generate a random prediction of the required length
        for j in df.index:
            sess_len = df.loc[j,'session_length']
            partial_len = math.ceil(sess_len/2)
            current_preds = np.random.choice([0, 1], size=(partial_len,))
            output.append(current_preds)
    return output

In [16]:
submission = generate_random_submission(input_logs)

In [7]:
def evaluate(submission,groundtruth):
    ap_sum = 0.0
    first_pred_acc_sum = 0.0
    counter = 0
    for sub, tru in zip(submission, groundtruth):
        if len(sub) != len(tru):
            raise Exception('Line {} should contain {} predictions, but instead contains '
                            '{}'.format(counter+1,len(tru),len(sub)))
        ap_sum += ave_pre(sub,tru,counter)
        first_pred_acc_sum += sub[0] == tru[0]
        counter+=1
    ap = ap_sum/counter
    first_pred_acc = first_pred_acc_sum/counter
    return ap,first_pred_acc

In [8]:
def ave_pre(submission,groundtruth,counter):
    s = 0.0
    t = 0.0
    c = 1.0
    for x, y in zip(submission, groundtruth):
        if x != 0 and x != 1:
            raise Exception('Invalid prediction in line {}, should be 0 or 1'.format(counter))
        if x==y:
            s += 1.0
            t += s / c
        c += 1
    return t/len(groundtruth)

In [9]:
ap,first_pred_acc = evaluate(submission,ground_truth)

In [10]:
print('average precision: {}'.format(ap))
print('first prediction accuracy: {}'.format(first_pred_acc))

average precision: 0.33620825485008854
first prediction accuracy: 0.5048
